<a href="https://colab.research.google.com/github/Vikram-Ahuja/Stock_Portfolios/blob/main/Pyhon_bot_freecodecamp_SPX500_EqualWeight_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [ ]:
stocks = pd.read_csv("/content/drive/MyDrive/sp_500_stocks.csv")
print(stocks["Ticker"])

In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448' #api token

In [ ]:
symbol = "APA"
api_url =f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

In [ ]:
price = data["latestPrice"]
market_cap = data["marketCap"]

In [ ]:
my_columns = ["Ticket" , "Price" ,"Market_Cap" ,"Stocks_to_buy"]
My_stock_dataframe = pd.DataFrame(columns=my_columns)
My_stock_dataframe

,Ticket,Price,Market_Cap,Stocks_to_buy


In [ ]:
'''
for stock in stocks["Ticker"][:20]:
    api_url =f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    My_stock_dataframe = My_stock_dataframe.append(
    pd.Series([
        stock,
        data["latestPrice"],
        data["marketCap"],
        "N/A"
    ],
    index = my_columns
    ),
    ignore_index= True
    )
'''

In [ ]:
def chunks(lst ,n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]

In [ ]:
symbol_grp = list(chunks(stocks["Ticker"],90))
symbol_string =[]
for i in range(0,len(symbol_grp)):
    symbol_string.append(",".join(symbol_grp[i]))

for symbol_str in symbol_string:
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_str}&types=quote&token={IEX_CLOUD_API_TOKEN}&format=json"
    Data = requests.get(batch_api_url).json()
    for sym in symbol_str.split(","):
        My_stock_dataframe = My_stock_dataframe.append(
            pd.Series([
                      sym,
                      Data[sym]["quote"]["latestPrice"],
                      Data[sym]["quote"]["marketCap"],
                      "N/A"
            ],
            index=my_columns
            ),
            ignore_index=True
        )

In [ ]:
Portfolio_size = float(input("Enter You Portfolio Size: "))
position_size = Portfolio_size/len(My_stock_dataframe)
for ind in My_stock_dataframe.index:
    no_stocks_to_buy = math.floor(position_size/My_stock_dataframe.Price[ind])
    My_stock_dataframe.Stocks_to_buy[ind] = no_stocks_to_buy
My_stock_dataframe.iloc[42]

In [ ]:
My_stock_dataframe.to_csv("/content/drive/MyDrive/Python_Portfolio.csv",index=False,header=True)